# Hbase in Python
## Requirements
- Hbase Installed
- Start Hbase: ./start-hase.sh
    - This will require the Zookeeper running, or in standalone mode the conf file specifications
- Start the Hbase thrift server
    - ./hbase thrift start